# Xакатон ЛИДЕРЫ ЦИФРОВОЙ ТРАНСФОРМАЦИИ 2023

## Северсталь. Модель раннего обнаружения неисправностей промышленного оборудования
[Задача 15](https://leaders2023.innoagency.ru/task_15) 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/YaninaK/predictive-maintenance.git -q
!pip install -r predictive-maintenance/requirements_Colab.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!git pull -q

In [1]:
%cd /content/predictive-maintenance

/content/predictive-maintenance


In [3]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "predictive_maintenance"))     

In [4]:
#import logging
import pandas as pd
import numpy as np
import pyspark
#from pyspark.sql import functions as F
#from functools import reduce

#from typing import Optional
from pprint import pprint

from data.make_dataset import (
    load_data, 
    get_new_X_column_names, 
    rename_columns,
    get_unified_tech_places,
)
from data.resample_dataset import (
    save_resampled_X,
    save_resampled_y_train,
    get_equipment_columns,
    resample,
)
from data.EDA_utilities import(
    add_groups_to_messages,
    identify_lack_of_messages_in_y_train,
    get_vectorizer_and_messages_vectors,
)
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
app_name = 'data_preprocessing'
spark_ui_port = 4041

In [7]:
spark = (
    pyspark.sql.SparkSession.builder
        .appName(app_name)        
        .config("spark.ui.port", spark_ui_port)
        .getOrCreate()
)

In [8]:
PATH = '/content/drive/MyDrive/ML_projects/predictive_maintenance/'

## 1. Data ingestion

In [9]:
X_train, y_train, messages, unified_tech_places = load_data(PATH)

X_train = X_train.repartition(4)
y_train = y_train.repartition(4)

print(f'X_train.shape = ({X_train.count()}, {len(X_train.columns)})')
print(f'y_train.shape = ({y_train.count()}, {len(y_train.columns)})')

X_train.shape = (9335034, 97)
y_train.shape = (9335034, 176)


In [10]:
messages.head(2)

,МАШИНА,ИМЯ_МАШИНЫ,ТЕХ_МЕСТО,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ,equipment,unified_name
390,AA2/006-006,ЭКСГАУСТЕР А/М №9,AA2/006-006-002-008,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,M3,неисправен двигатель,2019-01-21 00:00:00,2019-02-25,NaN,9,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №
391,CH-AGP-AG2/011-005,ЭКСГАУСТЕР А/М №9,CH-AGP-AG2/011-005-002,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,M3,неисправен двигатель,2019-01-21 12:26:08,2019-02-25,NaN,9,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №


In [11]:
unified_tech_places.head(2)

,equipment,description,unified_name
0,9,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №
1,9,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №


In [12]:
X_test = spark.read.parquet(
    PATH + 'data/01_raw/' + 'X_test.parquet', header=True, inferSchema= True
)
X_cols = get_new_X_column_names(X_test)
X_test = rename_columns(X_test, X_cols)
X_test = X_test.repartition(4)

print(f'X_test.shape = ({X_test.count()}, {len(X_test.columns)})')

X_test.shape = (4008961, 97)


In [13]:
test_intervals = pd.read_excel(
    PATH + 'data/01_raw/' + 'test_intervals.xlsx', index_col=0
).sort_index()

print(f'test_intervals.shape = {test_intervals.shape}')
test_intervals.head(2)

test_intervals.shape = (189, 4)


,start,finish,machine,tm
0,2022-01-07 09:05:16,2022-01-07 14:05:15,NaN,NaN
1,2022-02-25 03:44:52,2022-02-25 08:15:03,NaN,NaN


## 2. EDA

### 2.1 Target variable

#### 2.1.1. y_train

In [14]:
y_cols = get_equipment_columns(y_train.schema.names)

for i in range(4, 10): 
  print(len(y_cols[i]))

24
35
28
33
32
29


In [15]:
y_spec = pd.pivot_table(
    unified_tech_places, 
    index='unified_name', 
    columns='equipment', 
    values='description', 
    aggfunc='count',
    margins=True,    
)
print(f'y_spec.shape = {y_spec.shape}\n')
y_spec

y_spec.shape = (52, 7)



equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.1,1.0,1.0,NaN,NaN,NaN,1.0,3
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.2,NaN,1.0,1.0,1.0,1.0,NaN,4
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.3,NaN,1.0,1.0,1.0,1.0,NaN,4
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.4,NaN,NaN,NaN,1.0,1.0,NaN,2
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №,NaN,1.0,1.0,NaN,NaN,NaN,2
ГСМ ЭКСГ. №,1.0,1.0,NaN,1.0,1.0,1.0,5
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №,NaN,1.0,1.0,NaN,NaN,1.0,3
ДВИГАТЕЛЬ РЕЗЕРВНОГО МАСЛОНАСОСА ЭКСГ.№,NaN,1.0,NaN,NaN,NaN,NaN,1
ЗАДВИЖКА ЭКСГ. №,1.0,1.0,1.0,1.0,1.0,1.0,6


Информация об остановках или аномалиях доступна менее, чем по 60% тех.мест.

In [16]:
y_train.select(y_cols[4]).orderBy("DT").show(5) 

+-------------------+------------------------------------------+--------------+-------------------+------------------------------+-------------------------------+-------------------------------+-----------------------------+----------------------------------+----------------------+-----------------------------------+----------------------------+-------------------------------+-------------------------------+---------------------------------+----------------+------------------------------------------+----------------------------+---------------------------------------+----------------+-------------------+-------------------------------------+----------------------------+-----------------------------------------+
|                 DT|4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№4 Т_1|4_ГСМ ЭКСГ_ №4|4_ЗАДВИЖКА ЭКСГ_ №4|4_ЗАП_ И РЕГ_ АРМАТУРА ЭКСГ_№4|4_КЛ1 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4|4_КЛ2 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4|4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ_ №4|4_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ_ №4|4_МАСЛОПРОВОДЫ ЭКСГ 

In [17]:
i = 4
(
    resample(y_train.select(y_cols[i]))
    .groupBy("dt_resampled")
    .max()
    .orderBy("dt_resampled")
    .show(15)
)

+-------------------+----------+-----------------------------------------------+-------------------+------------------------+-----------------------------------+------------------------------------+------------------------------------+----------------------------------+---------------------------------------+---------------------------+----------------------------------------+---------------------------------+------------------------------------+------------------------------------+--------------------------------------+---------------------+-----------------------------------------------+---------------------------------+--------------------------------------------+---------------------+------------------------+------------------------------------------+---------------------------------+----------------------------------------------+
|       dt_resampled|max(epoch)|max(4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№4 Т_1)|max(4_ГСМ ЭКСГ_ №4)|max(4_ЗАДВИЖКА ЭКСГ_ №4)|max(4_ЗАП_ И РЕГ_ АРМАТУРА Э

10-секундный интервал между измерениями увеличен до 30-минутного. По y-train взято максимальное значение на интервале.

In [18]:
save = True
if save:
  save_resampled_y_train(y_train, path=PATH)

In [19]:
i = 4
df = pd.read_parquet(PATH + "data/02_intermediate/" + f'y{i}_resampled.parquet')
df[df.iloc[:, 2:].sum(axis=1) == 0]

,dt_resampled,max(epoch),max(4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№4 Т_1),max(4_ГСМ ЭКСГ_ №4),max(4_ЗАДВИЖКА ЭКСГ_ №4),max(4_ЗАП_ И РЕГ_ АРМАТУРА ЭКСГ_№4),max(4_КЛ1 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4),max(4_КЛ2 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4),max(4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ_ №4),max(4_МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ_ №4),...,max(4_РЕДУКТОР ГАЗ_ ЗАДВИЖКИ ЭКСГ_ №4),max(4_РОТОР ЭКСГ_ №4),max(4_ТИРИСТ_ ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ4 ВУ1),max(4_ТР-Р ТМ-4000-10/6 ЭКСГ_ №4),max(4_ТСМТ-101-010-50М-400 ТЕРМОПР_ПОДШ_Т_1),max(4_УЛИТА ЭКСГ_ №4),max(4_ЭКСГАУСТЕР А/М №4),max(4_ЭЛ/ДВИГАТЕЛЬ ГАЗ_ ЗАДВИЖКИ ЭКСГ_ №4),max(4_ЭЛЕКТРОАППАРАТУРА ЭКСГ_ №4),max(4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ_ №4)
0,2019-01-16 13:00:00,1547643600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-16 13:30:00,1547645400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-16 14:00:00,1547647200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-16 14:30:00,1547649000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-16 15:00:00,1547650800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2019-01-26 21:30:00,1548538200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,2019-01-26 22:00:00,1548540000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,2019-01-26 22:30:00,1548541800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,2019-01-26 23:00:00,1548543600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


По 4 эксгаустеру доступно 502 получасовых интервала, когда оборудование работало штатно.

#### 2.1.2. Messages

In [20]:
messages['ВИД_СООБЩЕНИЯ'].value_counts()

M3    898
M1     83
Name: ВИД_СООБЩЕНИЯ, dtype: int64

In [21]:
messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ'].min(), messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ'].max()

(Timestamp('2019-01-21 00:00:00'), Timestamp('2021-12-29 16:27:02'))

In [22]:
messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'].min(), messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'].max()

(Timestamp('2019-01-22 00:00:00'), Timestamp('2021-12-30 00:00:00'))

In [23]:
(messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'] - messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ']).describe()

count                           851
mean     49 days 23:01:29.286721504
std      90 days 01:42:27.596829132
min                 0 days 00:00:00
25%          0 days 07:32:29.500000
50%                12 days 21:28:47
75%         65 days 20:37:10.500000
max               728 days 15:13:55
dtype: object

In [24]:
messages = add_groups_to_messages(messages)

messages_spec_1 = pd.pivot_table(
    messages, 
    index='groups', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,
)
messages_spec_1

equipment,4,5,6,7,8,9,All
groups,,,,,,,
bearings,52.0,53.0,29.0,93.0,53.0,38.0,318
electric_devices,20.0,38.0,10.0,21.0,19.0,12.0,120
fittings,88.0,68.0,82.0,59.0,61.0,39.0,397
oil_system,13.0,38.0,13.0,23.0,28.0,22.0,137
other,2.0,2.0,1.0,3.0,1.0,NaN,9
All,175.0,199.0,135.0,199.0,162.0,111.0,981


In [25]:
messages_spec_2, missing_messages = identify_lack_of_messages_in_y_train(
    messages, unified_tech_places
)
messages_spec_2

equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
All,175.0,199.0,135.0,199.0,162.0,111.0,981.0
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.1,1.0,2.0,NaN,NaN,NaN,1.0,4.0
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.2,NaN,3.0,NaN,1.0,2.0,NaN,6.0
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.3,NaN,4.0,NaN,4.0,2.0,NaN,10.0
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.4,NaN,NaN,NaN,1.0,2.0,NaN,3.0
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №,NaN,1.0,1.0,NaN,NaN,NaN,2.0
ГСМ ЭКСГ. №,5.0,5.0,NaN,5.0,6.0,3.0,24.0
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №,NaN,2.0,NaN,NaN,NaN,NaN,2.0
ДВИГАТЕЛЬ РЕЗЕРВНОГО МАСЛОНАСОСА ЭКСГ.№,NaN,NaN,NaN,NaN,NaN,NaN,NaN


40% всех сообщений приходится на 5 и 7 эксгаузеры. Между различными видами оборудования сообщения распределены очень неравномерно.

In [26]:
missing_messages

equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.2,0,0,1,0,0,0,1
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.3,0,0,1,0,0,0,1
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №,0,0,1,0,0,1,2
ДВИГАТЕЛЬ РЕЗЕРВНОГО МАСЛОНАСОСА ЭКСГ.№,0,1,0,0,0,0,1
КЛ1 ТР№ ДО ЭД ЭКСГАУСТЕРА №,0,0,1,1,0,0,2
КЛ2 ТР№ ДО ЭД ЭКСГАУСТЕРА №,0,0,1,0,0,0,1
МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№,0,0,1,0,0,0,1
МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №,1,0,1,0,0,1,3
МЕТРАН-100 ДАТЧИКИ ДАВЛЕНИЯ ЭКСГ.№,0,0,0,1,0,1,2


Единицами отмечены тех.места соответствующих эксгаустеров из y_train, о которых  отсутствует информация в messages.

In [27]:
missing_messages.sum(axis=0)

equipment
4       1
5       4
6       7
7       6
8       0
9       7
All    25
dtype: int64

По 25 видам оборудования y_train фиксирует нештатные ситуации, но информации в messages по ним нет.

##### A. Stoppage M1

In [28]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M1'], 
    index='ТЕКСТ_ГРУППЫ_КОДОВ', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,
)

equipment,4,5,6,7,8,9,All
ТЕКСТ_ГРУППЫ_КОДОВ,,,,,,,
ВНЕШНИЕ ОРГАНИЗАЦИИ,NaN,1.0,NaN,NaN,1.0,NaN,2
ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ,17.0,19.0,6.0,11.0,12.0,4.0,69
ТЕХНИЧЕСКИЕ ПЛАНОВЫЕ,2.0,3.0,1.0,2.0,NaN,NaN,8
ТЕХНОЛОГИЧЕСКИЕ НЕПЛАНОВЫЕ,1.0,NaN,NaN,1.0,1.0,NaN,3
ТЕХНОЛОГИЧЕСКИЕ ПЛАНОВЫЕ,NaN,NaN,1.0,NaN,NaN,NaN,1
All,20.0,23.0,8.0,14.0,14.0,4.0,83


83% остановок - технические неплановые.

In [29]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M1'], 
    index='groups', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,
)

equipment,4,5,6,7,8,9,All
groups,,,,,,,
bearings,9.0,9.0,NaN,5.0,7.0,NaN,30
electric_devices,4.0,9.0,NaN,3.0,5.0,NaN,21
fittings,5.0,3.0,7.0,3.0,1.0,4.0,23
other,2.0,2.0,1.0,3.0,1.0,NaN,9
All,20.0,23.0,8.0,14.0,14.0,4.0,83


Больше половины остановок М1 прихходилось на 4 и 5 эксгаузеры.

In [30]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M1'], 
    index='unified_name', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,
)

equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №,NaN,1.0,1.0,NaN,NaN,NaN,2
ЗАДВИЖКА ЭКСГ. №,4.0,1.0,6.0,NaN,NaN,NaN,11
КОРПУС ЭКСГ. №,NaN,NaN,NaN,2.0,NaN,NaN,2
ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №,1.0,NaN,NaN,NaN,NaN,NaN,1
ПОДШИПНИК ОПОРНЫЙ №1,NaN,5.0,NaN,NaN,NaN,NaN,5
ПОДШИПНИК ОПОРНЫЙ №2,3.0,NaN,NaN,1.0,NaN,NaN,4
РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №,1.0,NaN,NaN,NaN,NaN,NaN,1
РОТОР ЭКСГ. №,5.0,4.0,NaN,4.0,7.0,NaN,20
ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ ВУ1,NaN,4.0,NaN,NaN,NaN,NaN,4


Причиной четверти всех остановок М1 был ротор.

##### B. Anomalies M3

In [31]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M3'], 
    index='groups', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,
)

equipment,4,5,6,7,8,9,All
groups,,,,,,,
bearings,43,44,29,88,46,38,288
electric_devices,16,29,10,18,14,12,99
fittings,83,65,75,56,60,35,374
oil_system,13,38,13,23,28,22,137
All,155,176,127,185,148,107,898


In [32]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M3'], 
    index='unified_name', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,
)

equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.1,1.0,2.0,NaN,NaN,NaN,1.0,4
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.2,NaN,3.0,NaN,1.0,2.0,NaN,6
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.3,NaN,4.0,NaN,4.0,2.0,NaN,10
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№ Т.4,NaN,NaN,NaN,1.0,2.0,NaN,3
ГСМ ЭКСГ. №,5.0,5.0,NaN,5.0,6.0,3.0,24
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №,NaN,2.0,NaN,NaN,NaN,NaN,2
ЗАДВИЖКА ЭКСГ. №,12.0,22.0,12.0,8.0,12.0,2.0,68
ЗАП. И РЕГ. АРМАТУРА ЭКСГ.№,1.0,NaN,NaN,NaN,NaN,NaN,1
ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №,NaN,3.0,NaN,NaN,2.0,1.0,6


Наибольшее число аномалий М3 зафиксировано на эксгаузерах 5 и 7.

##### C. Text vectorizing

In [33]:
vectorizer, X =  get_vectorizer_and_messages_vectors(messages)
print(f'X.shape = {X.shape}\n')

dict_ = sorted(vectorizer.vocabulary_, key=vectorizer.vocabulary_.get, reverse=True)
for i in dict_:
  print(i, vectorizer.vocabulary_[i])

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


X.shape = (981, 150)

электродвигатель 149
эл 148
эксгаустер 147
эксг ра 146
эксг 145
экс ра 144
экс 143
щетка 142
шток 141
шпилька 140
часть 139
фланец 138
утечка смазка 137
утечка масло 136
утечка 135
уровень масло 134
уровень 133
уплотнение разъем 132
уплотнение 131
улита 130
трансформатор 129
тр 128
течь масло 127
течь 126
температура 125
статор 124
смазка 123
сильный 122
сигнал 121
сгорать электродвигатель 120
сгорать 119
рп 118
ротор 117
резьба 116
редуктор 115
расцентровка 114
разъем улита 113
разъем 112
рабочий 111
работать 110
ра температура 109
ра вибрация 108
ра 107
просос разъем 106
просос 105
примесь 104
показание температура 103
показание вибрация 102
показание 101
подшипник обмотка 100
подшипник 99
подтекать масло 98
подтекать 97
подклинивать 96
подача 95
повышенный 94
патрубок 93
палец 92
отсутствовать 91
открытие 90
остановка эксгаустер 89
остановка эксг 88
остановка ра 87
остановка 86
ослабнуть 85
ослабление 84
обрыв 83
обмотка статор 82
обмотка 81
низкий уровень 80
н

### 2.2. X_train

In [34]:
X_cols = get_equipment_columns(X_train.schema.names)

In [35]:
X_train.select(X_cols[6]).orderBy("DT").show(5) 

+-------------------+---------------------+---------------------+---------------------+--------------------------------+---------------------+--------------------------------+--------------------------+--------------------------------+-----------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+------------------+------------------+------------------+
|                 DT|6 ВИБРАЦИЯ НА ОПОРЕ 1|6 ВИБРАЦИЯ НА ОПОРЕ 2|6 ВИБРАЦИЯ НА ОПОРЕ 3|6 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ|6 ВИБРАЦИЯ НА ОПОРЕ 4|6 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ|6 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ|6 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ|6 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4|    6 ТОК РОТОРА 1|    6 ТОК РОТОРА 2|     6 ТОК СТАТОРА|
+-------------------+---------------------+---------------------+-

In [36]:
i = 4
(
    resample(X_train.select(X_cols[i]), period=60*30)
    .groupBy("dt_resampled")
    .mean()
    .orderBy("dt_resampled")
    .show(15)
)

+-------------------+-----------+--------------------------+--------------------------+--------------------------+-------------------------------------+--------------------------+-------------------------------------+-------------------------------+-------------------------------------+----------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+-------------------+-------------------+------------------+
|       dt_resampled| avg(epoch)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 1)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 2)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 3)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 4)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ)|avg(4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ)|avg(4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ)|avg(4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ)|avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1)|avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2)|avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3)|avg(4 ТЕМ

In [37]:
save = True
if save:
  save_resampled_X(X_train, prefix="X_train", path=PATH)

In [38]:
i = 4
df = pd.read_parquet(
    PATH + "data/02_intermediate/" + f'X_train{i}_mean_resampled.parquet'
)
df.head()

,dt_resampled,avg(epoch),avg(4 ВИБРАЦИЯ НА ОПОРЕ 1),avg(4 ВИБРАЦИЯ НА ОПОРЕ 2),avg(4 ВИБРАЦИЯ НА ОПОРЕ 3),avg(4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ),avg(4 ВИБРАЦИЯ НА ОПОРЕ 4),avg(4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ),avg(4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ),avg(4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ),avg(4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ),avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1),avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2),avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3),avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4),avg(4 ТОК РОТОРА 1),avg(4 ТОК РОТОРА 2),avg(4 ТОК СТАТОРА)
0,2019-01-16 13:00:00,1.547644e+09,1.555479,0.734822,0.652846,1.609719,3.297655,0.723140,119.338196,40.327572,33.363486,41.469402,39.766466,39.926317,44.673509,362.470365,362.470365,242.106509
1,2019-01-16 13:30:00,1.547645e+09,1.561917,0.738544,0.642654,1.629317,3.260738,0.752766,119.472094,40.326196,33.330226,41.458176,39.716148,39.958300,44.639066,360.290962,360.290962,245.530294
2,2019-01-16 14:00:00,1.547647e+09,1.560495,0.742096,0.637606,1.536847,2.505211,0.722980,119.808137,40.181310,32.946006,41.367945,39.645747,39.884937,44.508887,356.424307,356.424307,244.586814
3,2019-01-16 14:30:00,1.547649e+09,1.552921,0.742581,0.644131,1.597993,3.308348,0.743605,119.716988,40.155692,33.282268,41.368633,39.600247,39.850154,44.572482,357.412598,357.412598,244.966794
4,2019-01-16 15:00:00,1.547651e+09,1.547022,0.732789,0.641007,1.636592,3.374121,0.765043,119.512763,40.228135,33.712776,41.389879,39.618564,39.916285,44.725751,358.037421,358.036736,246.133760


In [39]:
df.describe(percentiles=[0.25, 0.5, 0.75, 0.99]).T

,count,mean,std,min,25%,50%,75%,99%,max
avg(epoch),51862.0,1.594318e+09,2.694855e+07,1.547644e+09,1.570981e+09,1.594318e+09,1.617656e+09,1.640060e+09,1.640993e+09
avg(4 ВИБРАЦИЯ НА ОПОРЕ 1),50601.0,1.252102e+00,4.904377e-01,0.000000e+00,9.180708e-01,1.347995e+00,1.564374e+00,2.209234e+00,3.066241e+00
avg(4 ВИБРАЦИЯ НА ОПОРЕ 2),50614.0,9.307013e-01,3.730291e-01,0.000000e+00,6.999708e-01,8.536385e-01,1.124756e+00,1.906169e+00,2.974146e+00
avg(4 ВИБРАЦИЯ НА ОПОРЕ 3),50601.0,1.174368e+00,8.585902e-01,0.000000e+00,4.964072e-01,8.609996e-01,1.734835e+00,3.650435e+00,5.146976e+00
avg(4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ),50802.0,3.093527e+00,1.871820e+00,0.000000e+00,1.753842e+00,2.549842e+00,4.265988e+00,8.064821e+00,1.280355e+01
avg(4 ВИБРАЦИЯ НА ОПОРЕ 4),50601.0,1.586873e+00,3.047845e+00,-1.000000e+02,1.074637e+00,1.449944e+00,2.186662e+00,4.002921e+00,7.681333e+00
avg(4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ),50674.0,1.218909e+00,7.942266e-01,0.000000e+00,5.749755e-01,1.028045e+00,1.691076e+00,3.271966e+00,5.840659e+00
avg(4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ),50018.0,1.310490e+02,2.663944e+01,0.000000e+00,1.154764e+02,1.295671e+02,1.484418e+02,1.852901e+02,3.398845e+02
avg(4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ),50530.0,4.098256e+01,5.915142e+00,0.000000e+00,3.779250e+01,4.165001e+01,4.466784e+01,5.235280e+01,5.571711e+01
avg(4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ),50569.0,3.563336e+01,7.994236e+00,-3.998734e-01,3.189313e+01,3.686015e+01,4.123573e+01,4.931946e+01,5.616831e+01


### 2.3. X_test

В наборе сигналов тестовой выборки (X_test) присутствуют интервалы с пропущенными значениями. Список интервалов предоставлен в test_intervals. Для этих интервалов необходимо предсказать наличие неисправности. 

In [40]:
X_test.select(X_cols[6]).orderBy("DT").show(5) 

+-------------------+---------------------+---------------------+---------------------+--------------------------------+---------------------+--------------------------------+--------------------------+--------------------------------+-----------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+--------------+--------------+-------------+
|                 DT|6 ВИБРАЦИЯ НА ОПОРЕ 1|6 ВИБРАЦИЯ НА ОПОРЕ 2|6 ВИБРАЦИЯ НА ОПОРЕ 3|6 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ|6 ВИБРАЦИЯ НА ОПОРЕ 4|6 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ|6 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ|6 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ|6 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3|6 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4|6 ТОК РОТОРА 1|6 ТОК РОТОРА 2|6 ТОК СТАТОРА|
+-------------------+---------------------+---------------------+---------------------+-----

In [41]:
i = 4
(
    resample(X_test.select(X_cols[i]), period=60*30)
    .groupBy("dt_resampled")
    .mean()
    .orderBy("dt_resampled")
    .show(15)
)

+-------------------+-----------+--------------------------+--------------------------+--------------------------+-------------------------------------+--------------------------+-------------------------------------+-------------------------------+-------------------------------------+----------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+-------------------+-------------------+------------------+
|       dt_resampled| avg(epoch)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 1)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 2)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 3)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 4)|avg(4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ)|avg(4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ)|avg(4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ)|avg(4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ)|avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1)|avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2)|avg(4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3)|avg(4 ТЕМ

In [42]:
save = True
if save:
  save_resampled_X(X_test, prefix='X_test', path=PATH)

### 2.4. test_intervals

Для этих интервалов необходимо предсказать наличие неисправности. В случае предсказанной неисправности, определить тип и время начала развития и наступления неисправности.

In [43]:
test_intervals.shape

(189, 4)

In [44]:
test_intervals['start'].min(), test_intervals['start'].max()

(Timestamp('2022-01-01 10:54:53'), Timestamp('2023-04-20 16:41:58'))

In [45]:
test_intervals['finish'].min(), test_intervals['finish'].max()

(Timestamp('2022-01-01 16:43:39'), Timestamp('2023-04-20 23:58:07'))

In [46]:
(test_intervals['finish'] - test_intervals['start']).describe()

count                          189
mean     0 days 04:59:35.428571428
std      0 days 03:06:42.587346539
min                0 days 02:42:34
25%                0 days 03:38:37
50%                0 days 04:13:31
75%                0 days 05:05:08
max                1 days 00:58:29
dtype: object

Тестовый период - с 01.01.2022 по 20.04.2023 включительно. В среднем, интервал предсказания составляет 5 часов, медиана - 4 часа 13 минут. Максимальный горизонт предсказания - 26 часов.